## 情報通信実験2 課題2
山田遼太

18B15997

情報通信実験2 (ICT.E218)

##### 協力者
- 栢之間諒汰
- 高嶋優希
- 木村優孝 
- 松浦寛和
- 野田夏海
- 下村杏華
- 安永真梨


#### 注意事項
- 正しくプログラムが動作しない、解答がどの問題に対応するか分からない、などの不備がございましたら、文脈上もっとも尤もらしい修正を施して採点していただけますと幸いです。
- プログラムに対するコメントはプログラムの下にも記述しています。クラスや関数がどのような役割で、処理の難所に関してはどのように処理しているかについても言及しています。
- 課題で要求された出力結果が提出された.ipynbに入っています。そのセルを実行するとその出力結果が消えてしまうのでもしも採点基準に出力結果があるのであれば、セルを実行する前に確認してください。

<span style="margin-left:8px; padding:6px 10px; background:#ff9933; color:#ffffff; font-weight:bold;border-radius:5px;-webkit-border-radius:5px;-moz-border-radius:5px;">２.１</span>

In [14]:
import numpy as np
import pprint

# charlistにnum進数の数字が入る。
# num進数の数字は、文字列に見立てている
def make_charlist(num, length):
    charlist = list()
    for i in range(num**length):
        charlist.append(np.base_repr(i, base=num))
        k = (length - len(charlist[i])) * "0"
        charlist[i] = k + charlist[i]
    return charlist

# 作ったnum進数のcharlistを順に取り出してsymbolの語に戻してる
# ついでなので確率もきたsymbolに対応する確率を掛け算することで同時に求めている
def join_symbol(charlist, symbols, probs):
    symbol = [""]*len(charlist); prob = [1.0]*len(charlist)
    for i, char in enumerate(charlist):
        for c in char:
            symbol[i] += str(symbols[int(c)])
            prob[i] *= probs[int(c)]
    return symbol, prob

def print_charcter(symbol, prob):
    for s, p in zip(symbol, prob):
        print('P('+ str(s)+') :', end='')
        print('{:.5g}'.format(p))

def block_code(num, length, symbols, probs):
    charlist = make_charlist(num, length)
    symbol, prob = join_symbol(charlist, symbols, probs)
    print_charcter(symbol, prob)
    
    
if __name__ == "__main__": 
    print('alphabet size> ', end='')
    num = int(input())

    print('length> ', end='')
    length = int(input())

    symbols = list()
    probs = list()
    for i in range(num):
        print('symbol_{}> '.format(i+1), end='')
        symbols.append(str(input()))
        print('p_{}> '.format(i+1), end='')
        probs.append(float(input()))
    
    block_code(num, length, symbols, probs)



alphabet size> 3
length> 4
symbol_1> a
p_1> 0.1
symbol_2> b
p_2> 0.2
symbol_3> c
p_3> 0.7
P(aaaa) :0.0001
P(aaab) :0.0002
P(aaac) :0.0007
P(aaba) :0.0002
P(aabb) :0.0004
P(aabc) :0.0014
P(aaca) :0.0007
P(aacb) :0.0014
P(aacc) :0.0049
P(abaa) :0.0002
P(abab) :0.0004
P(abac) :0.0014
P(abba) :0.0004
P(abbb) :0.0008
P(abbc) :0.0028
P(abca) :0.0014
P(abcb) :0.0028
P(abcc) :0.0098
P(acaa) :0.0007
P(acab) :0.0014
P(acac) :0.0049
P(acba) :0.0014
P(acbb) :0.0028
P(acbc) :0.0098
P(acca) :0.0049
P(accb) :0.0098
P(accc) :0.0343
P(baaa) :0.0002
P(baab) :0.0004
P(baac) :0.0014
P(baba) :0.0004
P(babb) :0.0008
P(babc) :0.0028
P(baca) :0.0014
P(bacb) :0.0028
P(bacc) :0.0098
P(bbaa) :0.0004
P(bbab) :0.0008
P(bbac) :0.0028
P(bbba) :0.0008
P(bbbb) :0.0016
P(bbbc) :0.0056
P(bbca) :0.0028
P(bbcb) :0.0056
P(bbcc) :0.0196
P(bcaa) :0.0014
P(bcab) :0.0028
P(bcac) :0.0098
P(bcba) :0.0028
P(bcbb) :0.0056
P(bcbc) :0.0196
P(bcca) :0.0098
P(bccb) :0.0196
P(bccc) :0.0686
P(caaa) :0.0007
P(caab) :0.0014
P(caac) :0.004

<span style="margin-left:8px; padding:6px 10px; background:#ff9933; color:#ffffff; font-weight:bold;border-radius:5px;-webkit-border-radius:5px;-moz-border-radius:5px;">２.２</span>

In [50]:
from heapq import *
from math import log2
from statistics import mean
import pprint
import numpy as np

class Table():
    def __init__(self, syms, code):
        self.syms = syms
        self.code = code
        
    def __len__(self):
        return len(self.syms)

class Code():
    def __init__(self, symbol, codeword, stack=""):
        self.symbol = symbol
        self.codeword = str(codeword)
        self.stack = stack
        
    # symbolをtableを用いてcodewordに変換
    def encode(self, table):
        # 左からself.stackに代入していき、マッチしたら置き換えている。
        for c in self.symbol:
            self.stack += c
            for i in range(len(table)):
                if self.stack == table.syms[i]:
                    self.codeword = self.codeword + table.code[i]
                    self.stack = ""


    def decode(self, table):
        # 左からself.stackに代入していき、マッチしたら置き換えている。
        for c in self.codeword:
            self.stack += c
            for i in range(len(table)):
                if self.stack == table.code[i]:
                    self.symbol = self.symbol + table.syms[i]
                    self.stack = ""

class Node(object):
    def __init__(self, value, prob, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        self.prob = prob

    def setChildren(self, l, r):
        self.left = l
        self.right = r

    def __repr__(self):
        return "%s -- %s -- %s _ %s" % (self.value, self.prob, self.left, self.right)

    def __cmp__(self, a):
        return cmp(self.prob, a.prob)
    # heap木でnode同士を比較するのでそれ用
    def __lt__(self,other):
            return (self.prob) < (other.prob)
codes = {}


def create_tree(node):
    # len　nodeが１になるまで回す。
    while len(node) > 1:
        # 確率の小さい方から二つを取り出す。heap木の利用。
        # 確率の小さいものほど下になるようにしている。
        l = heappop(node)
        r = heappop(node)
        # 取り出したやつの親のノードのコンストラクタを呼ぶ
        above = Node(None, r.prob + l.prob)
        above.setChildren(l, r)
        heappush(node, above)

def make_code(s, node):
    # sに文字列を完成させていく。
    # valueがあるのは葉だけ。
    if node.value:
        if not s:
            codes[node.value] = "0"
        else:
            codes[node.value] = s
    else:
        make_code(s+"0", node.left)
        make_code(s+"1", node.right)

# adapter
def huffman(syms, probs):
    # key=sym, value=probsの連想配列に変換してからNodeクラスの要素に入れる
    dic = {key: val for key, val in zip(syms, probs)}
    # nodesは葉の数だけできる。まだsetChildrenはしていない。
    nodes = [Node(s, p) for (s, p) in dic.items()]

    heapify(nodes)
    # 作ったnodeをcreate_treeへ
    create_tree(nodes)
    return make_code("", nodes[0])

def H(lst):
    return -sum([x*log2(x) if x != 0 else 0 for x in lst])

# charlistにnum進数の数字が入る。
# num進数の数字は、文字列に見立てている
def make_charlist(num, length):
    charlist = list()
    for i in range(num**length):
        charlist.append(np.base_repr(i, base=num))
        k = (length - len(charlist[i])) * "0"
        charlist[i] = k + charlist[i]
    return charlist

# 作ったcharlistを　つなげる
def join_symbol(charlist, symbols, probs):
    symbol = [""]*len(charlist); prob = [1.0]*len(charlist)
    for i, char in enumerate(charlist):
        for c in char:
            symbol[i] += str(symbols[int(c)])
            prob[i] *= probs[int(c)]
    return symbol, prob

def print_charcter(symbol, prob):
    for s, p in zip(symbol, prob):
        print('P( '+str(s)+"): "+str(p))

def block_code(num, length, symbols, probs):
    charlist = make_charlist(num, length)
    symbol, prob = join_symbol(charlist, symbols, probs)
    # print_charcter(symbol, prob)
    return symbol, prob



        


In [63]:
import pprint
if __name__ == "__main__": 
    print('alphabet size> ', end='')
    # length は３で固定
    length = 3
    num = int(input())

    symbols = list()
    probs = list()
    for i in range(num):
        print('symbol_{}> '.format(i+1), end='')
        symbols.append(str(input()))
        print('p_{}> '.format(i+1), end='')
        probs.append(float(input()))
    # まずはブロック化します。２−１と同じです。
    symbol, prob = block_code(num, length, symbols, probs)
    # 次にハフマン符号化します。
    # この流れがブロックハフマン符号化の流れです。
    huffman(symbol, prob)
    print('cw for ')
    pprint.pprint(codes)

    print('entropy: ')
    print(H(probs))
    
    print('symbols> ')
    syms = str(input())


    print('encode> ', end='')
    table = Table(list(codes.keys()), list(codes.values()))
    coding = Code(syms, "", "")
    coding.encode(table)
    print('encoded: ', end='')
    print(coding.codeword)
    codewords =coding.codeword
    print('symbols> ', end='')
    print(codewords)
    coding = Code("", codewords, "")
    table = Table(list(codes.keys()), list(codes.values()))
    coding.decode(table)
    print('decoded: ', end='')
    print(coding.symbol)


alphabet size> 4
symbol_1> a
p_1> 0.1
symbol_2> b
p_2> 0.3
symbol_3> c
p_3> 0.2
symbol_4> d
p_4> 0.4
cw for 
{'aaa': '010111010',
 'aab': '01011100',
 'aac': '010111011',
 'aad': '10001111',
 'aba': '00110000',
 'abb': '1100000',
 'abc': '11100110',
 'abd': '1111010',
 'aca': '100011100',
 'acb': '0011110',
 'acc': '10011100',
 'acd': '1011010',
 'ada': '10111000',
 'adb': '010001',
 'adc': '1000110',
 'add': '101010',
 'baa': '00110001',
 'bab': '1011101',
 'bac': '11100111',
 'bad': '001011',
 'bba': '1100001',
 'bbb': '01100',
 'bbc': '111000',
 'bbd': '11010',
 'bca': '0011001',
 'bcb': '101111',
 'bcc': '001101',
 'bcd': '00100',
 'bda': '001010',
 'bdb': '11011',
 'bdc': '111100',
 'bdd': '0001',
 'caa': '100011101',
 'cab': '0011111',
 'cac': '10011101',
 'cad': '1000101',
 'cba': '11100101',
 'cbb': '110001',
 'cbc': '001110',
 'cbd': '00000',
 'cca': '10111001',
 'ccb': '010110',
 'ccc': '1000100',
 'ccd': '100100',
 'cda': '0101111',
 'cdb': '01010',
 'cdc': '101100',
 'cdd':

<span style="margin-left:8px; padding:6px 10px; background:#ff9933; color:#ffffff; font-weight:bold;border-radius:5px;-webkit-border-radius:5px;-moz-border-radius:5px;">２.４</span>

In [18]:
import numpy as np
import struct

class Table():
    def __init__(self, symbols, codewords):
        self.symbols = symbols
        self.codewords = codewords
    

class Code():
    def __init__(self, symbol, codeword):
        self.symbol = symbol
        self.codeword = codeword
        
    def encode(self, table):
        # 各符号語の幅に対応するのが　segment
        segment = np.array([float(0), float(1)])
        # i文字目の符号帯域を同じ要領で作っていく
        for i in range(len(self.symbol)):
            width = segment[1]-segment[0]
            for j in range(len(table.symbols)):
                if self.symbol[i] == table.symbols[j]:
                    segment=[segment[0] + width * table.codewords[j][0], segment[0] + width * table.codewords[j][1]]
        width = segment[1] - segment[0]
        boarder = np.mean(segment)
        length = np.ceil(-np.log2(width)) + 1
        self.codeword = DoubleToBin(boarder)[1:int(length)]

    def decode(self, table, length):
        boarder = BinToDouble(self.codeword)
        for i in range(length):
            for j in range(len(table.symbols)):
                if float(table.codewords[j][0]) < boarder and boarder < float(table.codewords[j][1]):
                    self.symbol = self.symbol + table.symbols[j]
                    width = table.codewords[j][1] - table.codewords[j][0]
                    boarder=(boarder - table.codewords[j][0]) / width
                    break

# float型の10進数を２進数のfloatに変換してくれる関数
def DoubleToBin(f):
    full=str(bin(struct.unpack('>Q', struct.pack('>d', f))[0]))
    # 倍精度浮動小数点数型式のexponentの部分だけ取り出すように考えた
    zeronum=0b1111111111-int('0b'+full[2:12], 0)
    return '0'*zeronum+'1'+full[12:]

# その逆
def BinToDouble(f):
    zeronum=f.index('1')+1
    frac=(f[zeronum:]+'0'*52)[:52]
    # 倍精度浮動小数点数型式のexponentの部分だけ取り出すように考えた
    head=str(bin(0b1111111111-int(zeronum)))
    return struct.unpack('>d', struct.pack('>Q', int(head+frac, 0)))[0]

# 符号語を確率から求める
def make_codeword(probs):
    codewords = np.array([[0.0]*2]*len(probs))
    # 符号語の出現確率を一つ一つ取り出して信号ごとの符号帯域を確定させる
    for i, v in enumerate(probs):
        codewords[i, :]=[codewords[i-1,1], codewords[i-1,1] + v]
    return codewords


print('alphabet size ', end='')
n = int(input())
syms = list()
probs = list()
for i in range(n):
    print('symbol_{}> '.format(i+1), end='')
    syms.append(str(input()))
    print('p_{}> '.format(i+1), end='')
    probs.append(float(input()))

codewords = make_codeword(probs)
print(codewords)
print('symbols> ', end='')
symbol = str(input())
code = Code(symbol, "")
table = Table(syms, codewords)
code.encode(table)
print('encode: ', end='')
print(code.codeword)


print('codewords> ', end='')
codeword = str(input())
print('length> ', end='')
length=int(input())
code = Code("", codeword)
table = Table(syms, codewords)
code.decode(table, length)
print('decode: ', end='')
print(code.symbol)

alphabet size 4
symbol_1> a
p_1> 0.6
symbol_2> b
p_2> 0.2
symbol_3> c
p_3> 0.1
symbol_4> d
p_4> 0.1
[[0.  0.6]
 [0.6 0.8]
 [0.8 0.9]
 [0.9 1. ]]
symbols> aadaadaabcccbdbcbacc
encode: 010101100000010111011011011001101011111000000
codewords> 00101011001100110011
length> 30
decode: aaabdaaaaacaaaaaabababcbbbbadd


<span style="margin-left:8px; padding:6px 10px; background:#ff9933; color:#ffffff; font-weight:bold;border-radius:5px;-webkit-border-radius:5px;-moz-border-radius:5px;">２.６</span>

In [48]:
import numpy as np
import pprint

# charlistにnum進数の数字が入る。
# num進数の数字は、文字列に見立てている
def make_charlist(num, length):
    charlist = list()
    for i in range(num**length):
        charlist.append(np.base_repr(i, base=num))
        k = (length - len(charlist[i])) * "0"
        charlist[i] = k + charlist[i]
    return charlist

# 作ったnum進数のcharlistを順に取り出してsymbolの語に戻してる
# ついでなので確率もきたsymbolに対応する確率を掛け算することで同時に求めている
def join_symbol(charlist, symbols, probs, p_c):
    symbol = [""]*len(charlist); prob = [1.0]*len(charlist)
    for i, char in enumerate(charlist):
        for j, c in enumerate(char):
            symbol[i] += str(symbols[int(c)])
            # charlistのなかの文字列を順に取り出していくのが内側のfor
            # この文字列がくる確率を計算したいわけだが、一文字目だけprobs二文字目以降は条件付き確率
            if j == 0:
                prob[i] = probs[int(c)]
            else:
                prob[i] = float(prob[i]) * p_c[int(char[j-1])][int(char[j])]
    return symbol, prob

def print_charcter(symbol, prob):
    for s, p in zip(symbol, prob):
        print('P('+ str(s)+') :', end='')
        print('{:.5g}'.format(p))

def block_code(num, length, symbols, probs, p_c):
    charlist = make_charlist(num, length)
    symbol, prob = join_symbol(charlist, symbols, probs, p_c)
    print_charcter(symbol, prob)


if __name__ == "__main__":
    num = int(input("alphabet size: "))
    length = int(input("length: "))
    symbols = list()
    probs = list()
    p_c = [[0]*num]*num # 条件付き確率を格納
    for i in range(num):
        symbols.append(input("symbol_%d "% i))
        probs.append(input("p_%d "% i))
    for i in range(num):
        for j in range(num):
            p_c[i][j] = float(input("cp_%d%d: "% (i,j)))
    
    block_code(num, length, symbols, probs, p_c)


alphabet size: 2
length: 5
symbol_0 a
p_0 0.2
symbol_1 b
p_1 0.8
cp_00: 0.2
cp_01: 0.8
cp_10: 0.5
cp_11: 0.5
P(aaaaa) :0.0125
P(aaaab) :0.0125
P(aaaba) :0.0125
P(aaabb) :0.0125
P(aabaa) :0.0125
P(aabab) :0.0125
P(aabba) :0.0125
P(aabbb) :0.0125
P(abaaa) :0.0125
P(abaab) :0.0125
P(ababa) :0.0125
P(ababb) :0.0125
P(abbaa) :0.0125
P(abbab) :0.0125
P(abbba) :0.0125
P(abbbb) :0.0125
P(baaaa) :0.05
P(baaab) :0.05
P(baaba) :0.05
P(baabb) :0.05
P(babaa) :0.05
P(babab) :0.05
P(babba) :0.05
P(babbb) :0.05
P(bbaaa) :0.05
P(bbaab) :0.05
P(bbaba) :0.05
P(bbabb) :0.05
P(bbbaa) :0.05
P(bbbab) :0.05
P(bbbba) :0.05
P(bbbbb) :0.05


基本的には2-1と同じ問題です。  
しかし、今回違うのは条件付き確率が変わるということ。  
なので、各符号語のなかの文字の確率を掛け算するとき、最初の一文字目だけ分岐して確率を計算  
他は手前で出た文字列を考慮して条件付き確率を用いた。

### <span style="margin-left:8px; padding:6px 10px; background:#ff9933; color:#ffffff; font-weight:bold;border-radius:5px;-webkit-border-radius:5px;-moz-border-radius:5px;">２.７</span>

In [1]:
from heapq import *
from math import log2
from statistics import mean
import pprint
import numpy as np

class Table():
    def __init__(self, syms, code):
        self.syms = syms
        self.code = code
        
    def __len__(self):
        return len(self.syms)

class Code():
    def __init__(self, symbol, codeword, stack=""):
        self.symbol = symbol
        self.codeword = str(codeword)
        self.stack = stack
        
    # symbolをtableを用いてcodewordに変換
    def encode(self, table):
        # 左からself.stackに代入していき、マッチしたら置き換えている。
        for c in self.symbol:
            self.stack += c
            for i in range(len(table)):
                if self.stack == table.syms[i]:
                    self.codeword = self.codeword + table.code[i]
                    self.stack = ""


    def decode(self, table):
        # 左からself.stackに代入していき、マッチしたら置き換えている。
        for c in self.codeword:
            self.stack += c
            for i in range(len(table)):
                if self.stack == table.code[i]:
                    self.symbol = self.symbol + table.syms[i]
                    self.stack = ""

class Node(object):
    def __init__(self, value, prob, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        self.prob = prob

    def setChildren(self, l, r):
        self.left = l
        self.right = r

    def __repr__(self):
        return "%s -- %s -- %s _ %s" % (self.value, self.prob, self.left, self.right)

    def __cmp__(self, a):
        return cmp(self.prob, a.prob)
    # heap木でnode同士を比較するのでそれ用
    def __lt__(self,other):
            return (self.prob) < (other.prob)
codes = {}


def create_tree(node):
    # len　nodeが１になるまで回す。
    while len(node) > 1:
        # 確率の小さい方から二つを取り出す。heap木の利用。
        # 確率の小さいものほど下になるようにしている。
        l = heappop(node)
        r = heappop(node)
        # 取り出したやつの親のノードのコンストラクタを呼ぶ
        above = Node(None, r.prob + l.prob)
        above.setChildren(l, r)
        heappush(node, above)

def make_code(s, node):
    # sに文字列を完成させていく。
    # valueがあるのは葉だけ。
    if node.value:
        if not s:
            codes[node.value] = "0"
        else:
            codes[node.value] = s
    else:
        make_code(s+"0", node.left)
        make_code(s+"1", node.right)

# adapter
def huffman(syms, probs):
    # key=sym, value=probsの連想配列に変換してからNodeクラスの要素に入れる
    dic = {key: val for key, val in zip(syms, probs)}
    # nodesは葉の数だけできる。まだsetChildrenはしていない。
    nodes = [Node(s, p) for (s, p) in dic.items()]

    heapify(nodes)
    # 作ったnodeをcreate_treeへ
    create_tree(nodes)
    return make_code("", nodes[0])

def H(lst):
    return -sum([x*log2(x) if x != 0 else 0 for x in lst])

# charlistにnum進数の数字が入る。
# num進数の数字は、文字列に見立てている
def make_charlist(num, length):
    charlist = list()
    for i in range(num**length):
        charlist.append(np.base_repr(i, base=num))
        k = (length - len(charlist[i])) * "0"
        charlist[i] = k + charlist[i]
    return charlist

# 作ったcharlistを　つなげる
def join_symbol(charlist, symbols, probs):
    symbol = [""]*len(charlist); prob = [1.0]*len(charlist)
    for i, char in enumerate(charlist):
        for c in char:
            symbol[i] += str(symbols[int(c)])
            prob[i] *= probs[int(c)]
    return symbol, prob

def print_charcter(symbol, prob):
    for s, p in zip(symbol, prob):
        print('P( '+str(s)+"): "+str(p))

def block_code(num, length, symbols, probs):
    charlist = make_charlist(num, length)
    symbol, prob = join_symbol(charlist, symbols, probs)
    print_charcter(symbol, prob)



        


In [5]:
import pprint
if __name__ == "__main__": 
    print('alphabet size> ', end='')
    num = int(input())
    print('length> ', end='')
    length = int(input())
    

    symbols = list()
    probs = list()
    for i in range(num):
        print('symbol_{}> '.format(i+1), end='')
        symbols.append(str(input()))
        print('p_{}> '.format(i+1), end='')
        probs.append(float(input()))
    # まずはブロック化します。２−１と同じです。
    symbol, prob = block_code(num, length, symbols, probs)
    # 次にハフマン符号化します。
    # この流れがブロックハフマン符号化の流れです。
    huffman(symbol, prob)
    print('cw for ')
    pprint.pprint(codes)

    print('entropy: ')
    print(H(probs))

    print('symbols> ')
    syms = str(input())



    table = Table(list(codes.keys()), list(codes.values()))
    coding = Code(syms, "", "")
    coding.encode(table)
    print('encoded: ', end='')
    print(coding.codeword)
    codewords =coding.codeword
    print('symbols> ', end='')
    print(codewords)
    coding = Code("", codewords, "")
    table = Table(list(codes.keys()), list(codes.values()))
    coding.decode(table)
    print('decoded: ', end='')
    print(coding.symbol)
    


alphabet size> 2
length> 4
symbol_1> a
p_1> 0.8
symbol_2> b
p_2> 0.2
cw for 
{'aaaa': '0',
 'aaab': '1101',
 'aaba': '101',
 'aabb': '110011',
 'abaa': '100',
 'abab': '111111',
 'abba': '110010',
 'abbb': '11000111',
 'baaa': '1110',
 'baab': '111110',
 'baba': '111100',
 'babb': '1100010',
 'bbaa': '111101',
 'bbab': '1100001',
 'bbba': '1100000',
 'bbbb': '11000110'}
entropy: 
0.7219280948873623
symbols> 
aaababaaaaaaaaaabaaabbaa
encoded: 1101100001110111101
symbols> 1101100001110111101
decoded: aaababaaaaaaaaaabaaabbaa


![](18_15997-02-08.png)

<div style="height:12px;">
<span style="margin-left:8px; padding:6px 10px; background:#ff9933; color:#ffffff; font-weight:bold;border-radius:5px;-webkit-border-radius:5px;-moz-border-radius:5px;">２.８</span>
</div>





$ m(x) $の代わりに$ l(x) $を用いたとしても、
記号列$ x $が一意に特定できることを示したい。

ここで、記号列$ x $の区間はばつまり$ \alpha - \beta $ の半分が切り捨てられる大きさよりもつねに大きいことが言えればいい。


つまり、$
\frac{P(x)}{2} > L
$
を示したい。($ L $ は$ | m(x) - l(x) | $)

$$
\frac{P(x)}{2} - L > 0\\
L < 2^{-l(-x)}より\\
\frac{P(x)}{2} - 2^{-l(-x)} > 0 が言えればいい。\\
\frac{P(x)}{2} - 2^{-[-\log{2} P(x)] - 1} > 0\\
-x <= [-x] < -x + 1 が成り立つので\\
\frac{P(x)}{2} - 2^{\log{2} P(x) - 2} < 0 が言えれば十分。\\
P(x) - 2^{\log{2} P(x) - 2} < 0 \\
P(x) - \frac{P(x)}{2} < 0\\
\frac{P(x)}{2} < 0\\
$$

$0 < P(x) < 1$ より成り立ち、題意が示された。